In [ ]:
# %% Deep learning - Section 19.176
#    Classify Gaussian blurs

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [1]:
# %% Libraries and modules
import numpy                  as np
import matplotlib.pyplot      as plt
import torch
import torch.nn               as nn
import seaborn                as sns
import copy
import torch.nn.functional    as F
import pandas                 as pd
import scipy.stats            as stats
import sklearn.metrics        as skm
import time
import sys
import imageio.v2             as imageio
import torchvision
import torchvision.transforms as T

from torch.utils.data                 import DataLoader,TensorDataset,Dataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from scipy.stats                      import zscore
from sklearn.decomposition            import PCA
from scipy.signal                     import convolve2d
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')


In [ ]:
# %% Reminder of 2D Gaussian parameters

# G = exp( -(Ã^2 + Õ^2) / 2sig^2 )
# Ã = A - C_a
# Õ = O - C_o
#
# where A and O are 2D coords grids, C the center of locations, and sig is the
# full width at half maximum


In [54]:
# %% Generate data

# 2D Gaussian params
n_per_class = 1000
n_classes   = 2
img_size    = 91
x           = np.linspace(-4,4,img_size)
X,Y         = np.meshgrid(x,x)

widths      = [1.8,2.4]

# Preallocate tensors for images (N,channels,size,size) and labels (N)
images  = torch.zeros( n_classes*n_per_class,1,img_size,img_size )
labels  = torch.zeros( n_classes*n_per_class )

# Generate images
for i in range(n_classes*n_per_class):

    # Gaussian with random center offset (remainder trick for width, all even
    # images go into category 0, all odd images go into category 1)
    c = 2*np.random.randn(2)
    G = np.exp( -((X-c[0])**2 + (Y-c[1])**2 ) / (2*widths[i%2]**2) )

    # Layer some noise
    G = G + np.random.randn(img_size,img_size)/5

    # Add to tensor
    images[i,:,:,:] = torch.tensor(G).view(1,img_size,img_size)
    labels[i]       = i%2

labels = labels[:,None]


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5))/2
fig,axs = plt.subplots(3,7,figsize=(phi*7,7))

for i,ax in enumerate(axs.flatten()):

    pic = np.random.randint(2*n_per_class)
    G   = np.squeeze( images[pic,:,:] )

    ax.imshow(G,vmin=-1,vmax=1,cmap='jet')
    ax.set_title('Class %s'%int(labels[pic].item()))
    ax.set_xticks([])
    ax.set_yticks([])

plt.savefig('figure8_classify_gaussian_blurs.png')
plt.show()
files.download('figure8_classify_gaussian_blurs.png')


In [56]:
# %% Create train and test datasets

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(images,labels,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [ ]:
# %% Check sizes

# Should be (N,channels,width,height) and (N)
print( train_loader.dataset.tensors[0].shape )
print( train_loader.dataset.tensors[1].shape )


In [84]:
# %% Function to generate the model

# Combine custom class and nn.Sequential
def gen_model():

    class Gauss_CNN(nn.Module):
        def __init__(self):
            super().__init__()

            # Layers with nn.Sequential (activation function is treated as layer)
            self.model = nn.Sequential( nn.Conv2d(1,6,3,padding=1),  # out size: (91+2*1-3)/1 + 1 = 91
                                        nn.ReLU(),
                                        nn.AvgPool2d(2,2),           # out size: 91/2 = 45
                                        nn.Conv2d(6,4,3,padding=1),  # out size: (45+2*1-3)/1 + 1 = 45
                                        nn.ReLU(),
                                        nn.AvgPool2d(2,2),           # out size: 45/2 = 22
                                        nn.Flatten(),                # vectorise
                                        nn.Linear(22*22*4,50),       # out size: 50
                                        nn.Linear(50,1)              # out size: 1
                                        )

        def forward(self,x):
            return self.model(x)

    # Create model instance
    CNN = Gauss_CNN()

    # Loss function
    loss_fun = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer


In [ ]:
# %% Test the model on one batch

CNN,loss_fun,optimizer = gen_model()

X,y  = next(iter(train_loader))
yHat = CNN(X)

# Check sizes of output and target variable
print()
print(yHat.shape), print()
print(y.shape), print()

# Check loss
loss = loss_fun(yHat,y)
print(loss)


In [ ]:
# %% Check all the parameters in the model

summary(CNN,(1,img_size,img_size))


In [46]:
# %% Function to train the model

def train_model():

    # Parameters, model instance, inizialise vars
    num_epochs = 10
    CNN,loss_fun,optimizer = gen_model()

    train_losses = []
    test_losses  = []
    train_acc    = []
    test_acc     = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = CNN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())
            batch_acc.append( torch.mean( ((yHat>0)==y).float() ).item() )

        train_losses.append( np.mean(batch_loss) )
        train_acc.append( 100*np.mean(batch_acc) )

        # Test accuracy
        CNN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = CNN(X)
            loss = loss_fun(yHat,y)

            test_acc.append( 100*torch.mean( ((yHat>0)==y).float() ).item() )
            test_losses.append(loss.item())

        CNN.train()

    return train_acc,test_acc,train_losses,test_losses,CNN


In [100]:
# %% Run the model

# Takes ~30 secs
train_acc,test_acc,train_losses,test_losses,CNN = train_model()


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,ax = plt.subplots(1,2,figsize=(1.5*phi*6,6))

ax[0].plot(train_losses,'s-',label='Train')
ax[0].plot(test_losses,'o-',label='Test')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss (MSE)')
ax[0].set_title('Model loss')

ax[1].plot(train_acc,'s-',label='Train')
ax[1].plot(test_acc,'o-',label='Test')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy (%)')
ax[1].set_title(f'Final model test accuracy: {test_acc[-1]:.2f}%')
ax[1].legend()

plt.savefig('figure9_classify_gaussian_blurs.png')
plt.show()
files.download('figure9_classify_gaussian_blurs.png')


In [ ]:
# %% Plotting

# Pass test data through model
X,y  = next(iter(test_loader))
yHat = CNN(X)

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(2,10,figsize=(2*phi*5,5))

for i,ax in enumerate(axs.flatten()):

    G = torch.squeeze( X[i,0,:,:] ).detach()
    ax.imshow(G,vmin=-1,vmax=1,cmap='jet')
    t = ( int(y[i].item()), int(yHat[i].item()>0) )

    ax.set_title('T=%s, P=%s'%t)
    ax.set_xticks([])
    ax.set_yticks([])

plt.savefig('figure10_classify_gaussian_blurs.png')
plt.show()
files.download('figure10_classify_gaussian_blurs.png')


In [ ]:
# %% Explore kernels

# Grab some filters
print(CNN), print()

layer_1_k = CNN.model[0].weight
layer_3_k = CNN.model[3].weight

print(layer_1_k.shape)
print(layer_3_k.shape)


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(1,6,figsize=(2*phi*6,6))

for i,ax in enumerate(axs.flatten()):

    ax.imshow( torch.squeeze(layer_1_k[i,:,:,:]).detach() ,cmap='plasma')
    ax.axis('off')

plt.suptitle('First convolution layer filters')

plt.savefig('figure11_classify_gaussian_blurs.png')
plt.show()
files.download('figure11_classify_gaussian_blurs.png')


In [ ]:
# %% Plotting

phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(4,6,figsize=(2*phi*6,6))

for i in range(6*4):

    idx = np.unravel_index(i,(4,6))
    axs[idx].imshow( torch.squeeze(layer_3_k[idx[0],idx[1],:,:]).detach() ,cmap='plasma')
    axs[idx].axis('off')

plt.suptitle('Second convolution layer filters')

plt.savefig('figure12_classify_gaussian_blurs.png')
plt.show()
files.download('figure12_classify_gaussian_blurs.png')


In [81]:
# %% Exercise 1
#    Rewrite the model architecture without using nn.Sequential. Your final result must be the same as the current version,
#    just defined in a different way. This is great practice at constructing models using classes.

# %% Function to generate the model

def gen_model():

    class mnist_CNN(nn.Module):
        def __init__(self):
            super().__init__()

            # Convolution layer 1
            # size = np.floor( (91+2*1-3)/1 )+1 = 91 (divide by 2 because will
            # have maxpool with extent 2)
            self.conv1 = nn.Conv2d(1,6,kernel_size=3,stride=1,padding=1)

            # Convolution layer 2
            # size = np.floor( (45+2*1-3)/1 )+1 = 45 (divide by 2 because will
            # have maxpool with extent 2)
            self.conv2 = nn.Conv2d(6,4,kernel_size=3,stride=1,padding=1)

            # Number of units expected in fully connected layer (out of conv2);
            # note that fc1 layer has no padding nor kernel, so we set those
            # params to be 0 and 1 respectively; we can also square because the
            # images are squares
            expected_size = np.floor( 22+2*0-1 ) + 1
            expected_size = 4*int(expected_size**2)

            # Fully connected layer
            self.fc1 = nn.Linear(expected_size,50)

            # Output layer
            self.output = nn.Linear(50,1)

        def forward(self,x):

            # MaxPool and ReLu on convolution layer 1
            x = F.relu(F.avg_pool2d(self.conv1(x),2))

            # MaxPool and ReLu on convolution layer 2
            x = F.relu(F.avg_pool2d(self.conv2(x),2))

            # Vectorise for linear layer
            x = torch.flatten(x,start_dim=1)

            # Linear and output layers
            x = F.relu(self.fc1(x))
            x = self.output(x)

            return x

    # Create model instance
    CNN = mnist_CNN()

    # Loss function
    loss_fun = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer


In [ ]:
# %% Exercise 2
#    Find and plot the stimuli that the model guessed incorrectly. Is the correct answer obvious to you? Do the errors
#    tend to be obscured by the boundaries of the image, or is there any other reason you can find for why the model got
#    those wrong?

# Pass test data through model
X,y  = next(iter(test_loader))
yHat = CNN(X)

# Convert logits to predictions and find indices
preds     = (yHat > 0).float()
wrong_idx = torch.where(preds != y)[0]

print(f'Number of misclassified samples in this batch: {len(wrong_idx)}')

# If fewer than 4 mistakes exist, adjust automatically
n_plot = min(4, len(wrong_idx))

# Randomly choose misclassified samples
rand_idx = wrong_idx[torch.randperm(len(wrong_idx))[:n_plot]]


In [ ]:
# %% Exercise 2
#    Continue ...

# Plot
phi = (1 + np.sqrt(5)) / 2
fig,axs = plt.subplots(1,n_plot,figsize=(phi*6,6))

# If only 1 image, axs is not iterable
if n_plot == 1:
    axs = [axs]

for ax,i in zip(axs,rand_idx):

    G = torch.squeeze(X[i,0,:,:]).detach()
    ax.imshow(G,vmin=-1,vmax=1,cmap='jet')

    true_label = int(y[i].item())
    pred_label = int(preds[i].item())

    ax.set_title(f'T={true_label}, P={pred_label}')
    ax.set_xticks([])
    ax.set_yticks([])

plt.suptitle('Misclassified samples')
plt.tight_layout()

plt.savefig('figure14_classify_gaussian_blurs_extra2.png')
plt.show()
files.download('figure14_classify_gaussian_blurs_extra2.png')


In [99]:
# %% Exercise 3
#    Notice the choice of architecture here: 6 channels in the first convolution layer and 4 channels in the second. In
#    the lecture I said that CNNs typically get wider with each successive "convpool block." Does that mean that this
#    model is wrong? Or suboptimal? Think of some arguments for and against this organization. Then modify the code to
#    swap the widths (4 channels in the first conv layer and 6 channels in the second conv layer). Does that affect the
#    model's performance?

# I mean... so hard to say, even this "unusual" model works so well; probably
# this level of compression is more than enough for this simple dataset, where
# the only feature that has to be extracted is the difference in the gaussian
# spread. The modified model behaves very similarly, and the images that are
# more likely to be misclassified are still those that end up in the corner

# Function to generate the model
def gen_model():

    class Gauss_CNN(nn.Module):
        def __init__(self):
            super().__init__()

            # Layers with nn.Sequential (activation function is treated as layer)
            self.model = nn.Sequential( nn.Conv2d(1,4,3,padding=1),  # out size: (91+2*1-3)/1 + 1 = 91
                                        nn.ReLU(),
                                        nn.AvgPool2d(2,2),           # out size: 91/2 = 45
                                        nn.Conv2d(4,6,3,padding=1),  # out size: (45+2*1-3)/1 + 1 = 45
                                        nn.ReLU(),
                                        nn.AvgPool2d(2,2),           # out size: 45/2 = 22
                                        nn.Flatten(),                # vectorise
                                        nn.Linear(22*22*6,50),       # out size: 50
                                        nn.Linear(50,1)              # out size: 1
                                        )

        def forward(self,x):
            return self.model(x)

    # Create model instance
    CNN = Gauss_CNN()

    # Loss function
    loss_fun = nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = torch.optim.Adam(CNN.parameters(),lr=0.001)

    return CNN,loss_fun,optimizer
